In [12]:
import numpy as np
import gmpy2
from numpy.random import randint

### Ex. 3.4 Wiener’s attack
Given n = 317940011 and e = 77537081, apply Wiener’s attack in order to factor n. 

In [2]:
# calculating continued fraction expansion for nu/de
def cfe(nu, de):
    c = [int(nu/de)]
    while nu != 1:
        nu, de = de, nu
        ci = int(nu/de)
        c.append(ci)
        nu = nu%de
        # print(c)
    c.append(de)
    return c

In [3]:
# recover nu/de
def get_frac(c):
    c.reverse()
    # clist = []
    _nu, _de = 0, 1
    for i in c:
        nu, de = _nu, _de
        # clist.append([nu, de])
        # print(nu, de)
        _nu, _de = de, i * de + nu
    return (nu, de)

In [4]:
e = 77537081
n = 317940011
x = cfe(e, n)
c_lst = []

for i in range(len(x)):
    tmp = get_frac(x[0:i+1])
    c_lst.append(tmp)

Since $ed-t \varphi(n)=1$, first find all possible $\varphi(n) = (ed-1)/t$, and check if the equation $ x^{2}-(n-\varphi(n)+1) x+n=0 $ has integer solution.

$pq = n$ and $p+q = n-\varphi(n)+1$.

In [5]:
for i in c_lst:
    t, d = i
    if t:
        if (e*d - 1)%t == 0:
            varphi = int((e*d - 1)/t)
            p_q = n +1 - varphi
            delta = p_q*p_q - 4*n
            print('---------------')
            print(t, d, "p+q=", p_q, "varphi=", varphi)
            print(delta)
            if delta>=0:
                print(np.sqrt(delta))
            # print(np.sqrt(delta))

---------------
1 4 p+q= 7791689 varphi= 310148323
60709145712677
7791607.389536321
---------------
10 41 p+q= 37980 varphi= 317902032
170720356
13066.0
---------------
33252285 136350656 p+q= 1 varphi= 317940011
-1271760043


Therefore, $\Delta = 13066$ and $b = -(p+q) = -37980$.

We could find $p, q = \frac{-b \pm \Delta}{2}$

### Ex. 4 RSA

In [2]:
# Modular Exponentiation
# m is base, d is power, n is module
def mod_exp(m, d, n):
    x = 1
    while d:
        if d&1:
            x = x*m % n
        m = m * m % n
        d = d >> 1
    return x

# generate random number until a prime
def gen_rand(bit):
    while 1:
        num = gmpy2.mpz(1)
        a = randint(0, 2, bit)
        for i in a:
            num = num * 2 + i
        if gmpy2.is_prime(num):
            break
    return num

# set up security level
level = {80:1024, 112:2048, 128:3072, 192:7680, 256:15360}

In [7]:
# sel -- security level
# p, q -- gmpy objects
def generate(sel):
    if sel not in level:
        print("wrong security level")
        return
    p = gen_rand(level[sel])
    q = gen_rand(level[sel])
    return p, q

In [8]:
# p -- plaintext
# n -- generated large number
# e -- encryption key
def encrypt(p, n, e):
    c = mod_exp(p, e, n)
    return c

In [9]:
# c -- ciphertext
# n -- generated large number
# e -- decryption key
def decrypt(c, n, d):
    p = mod_exp(c, d, n)
    return p

In [20]:
p, q = generate(80)

In [22]:
n = p * q
print(n)

85251399824791625698367629049474620170498107429524969218263938529493969798084686229841677931184875261597898291409781527861054043998720967923487311723402629643151975676750596142935807384707225555880988617145476320769211145562713694456853347011642904524859843300279823453241124944547447866194570575089878546086094317193373462821276260012830824469420982084514841870188828598731620834073285594022208586078274805690331699591386287989128408736364169810602532793643209027815650503845372197796129728708223119465098966342155276977076870703574598539813061382744242834279509163112934436184488393968798018262070259293518100572363


### Ex. 5.5 smallest generator of $\varphi(97)$
$\varphi(97) = 96 = \times 6\times 16 = 3\cdot 2^5$, thus need to consider 32 and 48.

In [25]:
for k in [2, 3, 5, 7, 11]:
    print('---{}---'.format(k))
    print(mod_exp(k, 32, 97))
    print(mod_exp(k, 48, 97))

---2---
35
1
---3---
35
1
---5---
35
96
---7---
35
96
---11---
61
1


From the result we know that 5 is the smallest generator.

### Ex. 6.2 brutal force to find $x$ for $3^{x} \equiv 2 \bmod 65537$

In [26]:
tmp = mod_exp(3, 2048, 65537)
base = tmp * tmp % 65537
for i in range(15):
    if tmp == 2:
        print("x is", i*2+1)
        break
    tmp = tmp * base % 65537   # print(tmp)

x is 27


### Ex. 6.3 using Pohlig-Hellman algorithm

In [4]:
print(mod_exp(2, 2**15, 65537))
a_ = mod_exp(3, 2**15, 65537)
print(a_)

1
65536


Thus, with $2^{2^{15}} = (3^{2^{15}})^{x_0}$, we can find $x_0 = 0$.

In [5]:
b = 2
a = 3
x = [2]
for i in range(15):
    ex = 2**16-2**(i)*x[i] % 2**16
    b_ = b * 3**ex
    b = b_ % 65537
    bi = mod_exp(b, 2**(14-i), 65537)
    print(bi)
    tmp = 1
    xi = 0
    while True:
        if tmp==bi:
            break
        tmp = tmp * a_ % 65537
        print(tmp)
        xi = xi + 1
    x.append(xi)

65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
1
65536
65536
1
65536
65536
65536
65536


In [49]:
sum_ = 0
for i in range(16):
    sum_ = sum_ + x[i] * (2 ** i)

In [50]:
sum_ % 65537

55296

In [51]:
3**55296% 65537

2